# Présentation de la partie 2 de la SAE sur les graphes

Dans cette seconde partie de SAE sur les graphes, l'objectif est de pouvoir faire de la recherche de plus courts chemins, en utilisant l'algorithme de Dijkstra. 
La recherche de plus courts chemins sera ensuite utilisée pour trouver les hôtels, restaurants et aire de covoiturage les plus proches d'une manifestation culturelle choisie.

La représentation choisie pour les graphes est une matrice d'adjacence, qui sera représentée par un tableau de tableau (les noms des sommets sont des entiers et seront numérotés en commençant à 0). Nous considérons que le nombre de sommets du graphe ne pourra pas être modifié une fois le graphe construit. Un tableau contenant le nom associé à chaque sommet sera également créé.


### Présentation du problème et choix d'une manifestation culturelle à étudier

L'office de tourisme de votre département propose une liste de restaurants et d'hôtels les plus proches d'une manifestation culturelle ainsi que l'aire de covoiturage la plus proche de celle-ci. Vous aurez à implémenter les fonctions permettant de créer cette liste.

Chaque binôme doit choisir la manifestation culturelle qu'il souhaite considérer et l'indiquer dans l'onglet correspondant du document partagé suivant : https://uncloud.univ-nantes.fr/index.php/s/BXwyNfYHbfKYbB8 (chaque binôme doit choisir parmi les manifestations culturelles du département assigné à son groupe).



## Installation de graphviz pour pouvoir visualiser les graphes


In [ ]:
%pip install graphviz
import graphviz
graphviz.__version__, graphviz.version()



## Création d'une fonction de conversion d'un graphe, défini par une matrice d'adjacence, vers le format graphviz


In [ ]:
def creation_graphe_graphviz(g):
    # création d'un graphe non orienté
    dot = graphviz.Graph()
    
    # ajout des sommets
    for i in range(len(g)):
        dot.node(str(i))
    
    # ajout des arcs
    for i in range(len(g)):
        for j in range(i):
            if g[i][j] == 1:
                dot.edge(str(i), str(j))
    
    return dot

## Fonctions utiles pour manipuler les graphes

#### Fonction qui construit un graphe vide (sans aretes), avec le nombre de sommets donné, et qui retourne la matrice d'adjacence correspondante

In [ ]:
def graphe_vide(n):
    g = []
    for i in range(n):
        l = []
        for j in range(n):
            l.append(0)
        g.append(l)
    return g

#### Fonction qui retourne vrai si l'arete, dont les numéros de sommets extrémités sont donnés, existe

In [ ]:
def est_arete(g, i, j):
    return g[i][j]

#### Fonction qui calcule les successeurs d'un sommet

In [ ]:
def successeurs(g, i):
    succs = []
    
    for x in g[i]:
        if x:
            succs.append(x)
    
    return succs

#### Fonction qui importe un graphe contenu dans un fichier, au format dot

In [ ]:
def importe_graphe(fich):
    graphe=graphviz.Source.from_file(fich)
    graphe.view()
    return graphe

In [ ]:
graphe=importe_graphe("grapheStablesCliques.dot")
print(graphe)

#### Fonction qui exporte un graphe dans un fichier, au format dot

In [ ]:
def exporte_graphe(fich, graphe, noms_sommets):
    dot = graphviz.Graph()  
    for i in range(len(graphe)):
        dot.node(str(i),str(noms_sommets[i]))
        for y in range(i,len(graphe)):
            if graphe[i][y]:
                dot.edge(str(i),str(y))
    dot.save(fich)
    return 

In [ ]:
# Test export

#nomssom=["A","B","C"]
#exporte_graphe("test.dot",graphe,nomssom)

## Calcul de plus courts chemins avec l'algorithme de Dijkstra

#### Fonction qui implémente l'algorithme de Dijkstra pour le calcul de plus courts chemins, à partir d'un sommet de départ s, et qui retourne le tableau des distances du sommet s aux autres sommets ainsi que le tableau des sommets prédécesseurs sur les plus courts chemins

In [ ]:
def algo_dijkstra(s, graphe):
    sommet_passe=[]
    dij=[]
    add=0
    while len(dij)<len(graphe):
        sommet_passe.append(s)
        dijs=[]
        for i in range(len(graphe[s])):
            if i in sommet_passe:
                dijs.append(0)
            elif graphe[s][i]==0:
                if len(dij)>0:
                    dijs.append(dij[-1][i])
                else:
                    dijs.append(0)
            else :
                dijs.append(graphe[s][i]+add)
        
        if len(dij)>0:
            for i in range(len(dijs)):
                if dijs[i]>dij[-1][i] and dij[-1][i]!=0:
                    dijs[i]=dij[-1][i]
                
        dij.append(dijs)
        if dijs.count(0)<len(dijs):
            add=sorted(dijs)[dijs.count(0)]
            s=dijs.index(add)
    dijlist=[]
    for i in range(len(dij)):
        max=0
        for y in range(len(dij[i])):
            if max < dij[y][i]:
                max=dij[y][i]
        dijlist.append(max)
    return dijlist

In [ ]:
# test de la fonction algo_dikstra avec un ou deux petits graphes
graphe2=[[0,1,2,0,0,0,0],
         [1,0,0,2,0,3,0],
         [2,0,0,3,4,0,0],
         [0,2,3,0,2,3,3],
         [0,0,4,2,0,0,5],
         [0,3,0,3,0,0,4],
         [0,0,0,3,5,4,0]]
algo_dijkstra(0,graphe2)

## Création de graphes de distances entre points

#### Installation de geopy pour calculer des distances à partir de coordonnées gps

In [1]:
%pip install geopy
import geopy
from geopy.distance import geodesic as gd
import pandas as pd

Note: you may need to restart the kernel to use updated packages.


#### Fonction qui crée un graphe de distance entre un sommet de départ, dont le nom et les coordonnées gps sont données (latitude et longitude), et des sommets contenus dans un fichier csv, pour un département choisi

In [2]:
def cree_graphe_distances(nom_sommet, lat_sommet, long_sommet, fich_csv, dpt):
    sommet=(lat_sommet, long_sommet)
    info_csv=[nom_sommet]
    data = pd.read_csv(fich_csv , sep=";")
    for i in data.departement:
        if i==dpt:
            long_csv=data["Longitude"]
            lat_csv=data["Latitude"]
            nom=data["Nom de l'offre touristique"]
            info1=data[data.columns[1]]
            info2=data[data.columns[2]]
            adr1=data["Adresse1"]
            adr2=data["Adresse2"]
            adr3=data["Adresse3"]


            list_coord=[sommet]
            for i in range(len(long_csv)):
                info_csv.append((nom[i],info1[i],info2[i],adr1[i],adr2[i],adr3[i]))
                list_coord.append((long_csv[i],lat_csv[i]))
    g=[]
    for y in list_coord:
        g.append(gd(list_coord[0],y).m)
    
                
    return info_csv,g        
            

In [ ]:
cree_graphe_distances("oui",77.0,88.0,"Donnees-csv/Donnees-csv/234400034_070-006_offre-touristique-hotels-rpdl.csv", "Loire-Atlantique")

## Calcul de plus courts chemins, en utilisant l'algorithme de Dijkstra implémenté 

Vous donnerez le nom et l'adresse des lieux à lister.

#### Quel est l'hôtel le plus proche de la manifestation culturelle choisie ?

In [6]:
info,g=cree_graphe_distances("L'APÉRO DU 14 JUILLET",48.097,-0.354,"Donnees-csv/Donnees-csv/234400034_070-006_offre-touristique-hotels-rpdl.csv", "Mayenne")
distance = [x for _,x in sorted(zip(g,info))]
distance[1]


('LE PRESSOIR HOTEL',
 '3 étoiles',
 'Hôtel - Restaurant',
 nan,
 '12 rue du champ de la croix',
 'Zone artisanale du Pressoir')

#### Quels sont les 10 hôtels les plus proches de la manifestation culturelle choisie ?

In [3]:
info,g=cree_graphe_distances("L'APÉRO DU 14 JUILLET",48.097,-0.354,"Donnees-csv/Donnees-csv/234400034_070-006_offre-touristique-hotels-rpdl.csv", "Mayenne")
distance = [x for _,x in sorted(zip(g,info))]
for i in range(1,11):
    print(distance[i])

('LE PRESSOIR HOTEL', '3 étoiles', 'Hôtel - Restaurant', nan, '12 rue du champ de la croix', 'Zone artisanale du Pressoir')
('HOTEL DE FRANCE', 'Non classé', 'Hôtel', nan, "5 place de l'Hôtel de ville", nan)
("HOTEL D'ANGLETERRE", '3 étoiles', 'Hôtel - Restaurant', nan, '9 rue du Guichet', nan)
('HOTEL LA GARGOTE', 'Non classé', 'Hôtel - Restaurant', nan, '2 rue du Lac', nan)
('LE SAINT PIERRE', 'Non classé', 'Hôtel - Restaurant', nan, '42 rue Nationale', 'Ruillé sur Loir')
('HÔTEL DE FRANCE', '3 étoiles', 'Hôtel - Restaurant', nan, '20, place de la République', nan)
('HOTEL LE RELAIS DU CHAPEAU ROUGE', 'Non classé', 'Hôtel - Restaurant', nan, "6 place de l'Hôtel de Ville", nan)
('HOTEL DE LA GARE', 'Non classé', 'Hôtel - Restaurant', nan, '170  avenue Jean Jaurès', 'Château du Loir')
('HOTEL-RESTAURANT LOGIS LE GRAND HOTEL', '3 étoiles', 'Hôtel - Restaurant', nan, '59 rue Aristide Briand', 'Château du Loir')
("HÔTEL FONTEVRAUD L'HÔTEL", '4 étoiles', 'Hôtel - Restaurant', nan, 'Abbaye 

#### Quels sont les 10 hôtels 3 étoiles les plus proches de la manifestation culturelle choisie ?

In [4]:
info,g=cree_graphe_distances("L'APÉRO DU 14 JUILLET",48.097,-0.354,"Donnees-csv/Donnees-csv/234400034_070-006_offre-touristique-hotels-rpdl.csv", "Mayenne")
distance = [x for _,x in sorted(zip(g,info))]
x=10
i=0
while x>0:
    i+=1
    if distance[i][1]=="3 étoiles":
        print(distance[i])
        x-=1

('LE PRESSOIR HOTEL', '3 étoiles', 'Hôtel - Restaurant', nan, '12 rue du champ de la croix', 'Zone artisanale du Pressoir')
("HOTEL D'ANGLETERRE", '3 étoiles', 'Hôtel - Restaurant', nan, '9 rue du Guichet', nan)
('HÔTEL DE FRANCE', '3 étoiles', 'Hôtel - Restaurant', nan, '20, place de la République', nan)
('HOTEL-RESTAURANT LOGIS LE GRAND HOTEL', '3 étoiles', 'Hôtel - Restaurant', nan, '59 rue Aristide Briand', 'Château du Loir')
('HÔTEL LA CROIX BLANCHE FONTEVRAUD', '3 étoiles', 'Hôtel - Restaurant', nan, '5-7, place des Plantagenêts', nan)
('HÔTEL LE BUSSY', '3 étoiles', 'Hôtel', nan, "4 rue Jehanne d'Arc", nan)
('HÔTEL LE DOMAINE DE MESTRÉ', '3 étoiles', 'Hôtel - Restaurant', nan, 'Lieu dit Mestré', nan)
('HOTEL AKENA', '3 étoiles', 'Hôtel', 'RD 1 - Route de Saint Calais', nan, 'RD 1')
('AUBERGE DU CHEVAL BLANC', '3 étoiles', 'Hôtel - Restaurant', nan, '1, rue du Beau Soleil', nan)
('HÔTEL LA DEMEURE DE LA VIGNOLE', '3 étoiles', 'Hôtel', nan, '3 IMPASSE MARGUERITE D ANJOU', nan)


#### Quel est l'hôtel 4 étoile le plus proche de la manifestation culturelle choisie ?

In [5]:
info,g=cree_graphe_distances("L'APÉRO DU 14 JUILLET",48.097,-0.354,"Donnees-csv/Donnees-csv/234400034_070-006_offre-touristique-hotels-rpdl.csv", "Mayenne")
distance = [x for _,x in sorted(zip(g,info))]
x=1
i=0
while x>0:
    i+=1
    if distance[i][1]=="4 étoiles":
        print(distance[i])
        x-=1


("HÔTEL FONTEVRAUD L'HÔTEL", '4 étoiles', 'Hôtel - Restaurant', nan, 'Abbaye de Fontevraud', "38 rue St-Jean de l'Habit")
